#### ROUND 2:

In [1]:
from galactic_apis_gpt import GalaxyNavigator, GalacticMarketplace, InfoSphere

galaxy_state_file = "ROUND 2 FILES/galaxy_state_round2.json"

market = GalacticMarketplace(galaxy_state_file)
navigator = GalaxyNavigator()
infosphere = InfoSphere()

##### TASK 1: "Trova R2-D2 e portalo su Coruscant. Poi compra 2 Laser Sword diverse su pianeti diversi"

In [2]:
def find_droid(droid="R2-D2"):
    return navigator.find_asset_position(droid)

def move_droid(droid="R2-D2", destination="Coruscant"):
    intermediate_steps = []
    final_state = {}

    droid_location = find_droid(droid)
    intermediate_steps.append(f"Located {droid} at {droid_location}.")

    if destination == droid_location:
        agent_response = f"Droid {droid} already at {destination}."
        final_state = {
            "droid": droid,
            "location": droid_location,
            "status": "no_action_needed"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    try:
        travel_info = navigator.book_travel(droid_location, destination)
        if travel_info:
            intermediate_steps.append(f"Booked travel from {droid_location} to {destination}.")
            navigator.move_asset(droid, destination)
            intermediate_steps.append(f"Moved {droid} to {destination}.")
            agent_response = f"Travel successful: droid {droid} moved to {destination}."
            final_state = {
                "droid": droid,
                "location": destination,
                "status": "moved"
            }
            return {
                "agent_response": agent_response,
                "intermediate_steps": intermediate_steps,
                "final_state": final_state
            }
        else:
            agent_response = f"Travel not possible. Droid {droid} stuck at {droid_location}."
            final_state = {
                "droid": droid,
                "location": droid_location,
                "status": "stuck"
            }
            return {
                "agent_response": agent_response,
                "intermediate_steps": intermediate_steps,
                "final_state": final_state
            }
    except ValueError as e:
        intermediate_steps.append(f"Travel booking failed: {str(e)}")
        agent_response = f"Travel not possible due to insufficient funds. Droid {droid} stuck at {droid_location}."
        final_state = {
            "droid": droid,
            "location": droid_location,
            "status": "stuck",
            "error": str(e)
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

In [3]:
def task1(droid="R2-D2", target_item="Laser Sword"):
    intermediate_steps = []
    sword_counter = 0

    # Step 1: Trova la posizione iniziale del droide
    location = find_droid(droid)
    intermediate_steps.append(f"Droid '{droid}' located at {location}.")

    # Step 2: Cerca spade laser nel pianeta iniziale
    swords_on_current = market.find_items(location)
    intermediate_steps.append(f"Found {len(swords_on_current)} '{target_item}' on {location}.")

    purchased_items = []
    errors = []
    budget = navigator.get_client_budget()
    total_spent = 0

    # Prova subito ad acquistare una spada se presente
    if len(swords_on_current) > 0:
        try:
            purchase_result = market.purchase_item(swords_on_current[0]["id"])
            print(purchase_result["remaining_balance"])
            purchased_items.append({
                "item_id": swords_on_current[0]["id"],
                "name": target_item,
                "price": swords_on_current[0]["price"],
                "purchase_info": purchase_result
            })
            sword_counter += 1
            total_spent += swords_on_current[0]["price"]
            intermediate_steps.append(f"Purchased '{target_item}' on {location}.")
        except ValueError as e:
            errors.append(f"Could not purchase '{target_item}' on {location}: {str(e)}")

    # Step 3: Cerca su tutti i pianeti se necessario
    if sword_counter < 2:
        planets = infosphere.get_all_planets()
        planets = [planet for planet in planets if planet != location]
        intermediate_steps.append(f"Searching on planets: {', '.join(planets)}.")

        for planet in planets:
            if sword_counter >= 2:
                break

            if market.item_on_planet(target_item, planet):
                # Muovi droide
                print("Muoviamo il droide")
                move_result = move_droid(droid, planet)
                print(move_result)
                intermediate_steps.append(f"Moved droid to {planet}.")

                if move_result["final_state"]["status"] == "stuck":
                    errors.append(f"Could not move droid to {planet}.")
                    continue

                # Trova l'item di nuovo
                items_found = market.find_items(planet)
                if len(items_found) > 0:
                    try:
                        purchase_result = market.purchase_item(items_found[0]["id"])
                        purchased_items.append({
                            "item_id": items_found[0]["id"],
                            "name": target_item,
                            "price": items_found[0]["price"],
                            "purchase_info": purchase_result
                        })
                        sword_counter += 1
                        total_spent += items_found[0]["price"]
                        intermediate_steps.append(f"Purchased '{target_item}' on {planet}.")
                    except ValueError as e:
                        errors.append(f"Could not purchase '{target_item}' on {planet}: {str(e)}")

    # Step 4: Verifica esito
    if sword_counter < 2:
        agent_response = "Could not purchase two Laser Swords."
        status = "purchase_failed"
    else:
        agent_response = "Purchased 2 Laser Swords successfully."
        status = "success"

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Total swords purchased: {sword_counter}.",
            f"Total spent: {total_spent}."
        ],
        "final_state": {
            "status": status,
            "purchases": purchased_items,
            "errors": errors,
            "remaining_budget": navigator.get_client_budget()
        }
    }

In [4]:
#task1()

##### TASK 2: ""Usa l'InfoSfera per trovare informazioni su 'Alderaan'. Se il threat_level è 'low', trasporta R2-D2 lì""

In [5]:
def task2(droid="R2-D2", destination="Alderaan"):
    intermediate_steps = []

    # Step 1: Recupera informazioni sul pianeta
    info = infosphere.search_info(destination)
    threat_level = info.get("threat_level", "unknown")
    intermediate_steps.append(f"Retrieved threat level for {destination}: {threat_level}.")

    # Step 2: Controlla il livello di minaccia
    if threat_level != "low":
        agent_response = f"Could not move droid to {destination} due to threat level '{threat_level}'."
        final_state = {
            "status": "move_blocked",
            "threat_level": threat_level,
            "droid_location": find_droid(droid)
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    # Step 3: Muovi il droide
    move_result = move_droid(droid, destination)
    intermediate_steps.append(f"Droid moved to {destination}.")

    # Verifica se si è bloccato
    if move_result["final_state"]["status"] == "stuck":
        agent_response = f"Attempted to move droid to {destination}, but it got stuck."
        final_state = {
            "status": "move_failed",
            "droid_location": move_result["final_state"].get("current_location", "unknown")
        }
    else:
        agent_response = f"Droid moved to {destination} successfully."
        final_state = {
            "status": "success",
            "droid_location": destination
        }

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps,
        "final_state": final_state
    }

In [6]:
#task2()

##### TASK 3: "Ottimizza i costi: trova la rotta più economica per visitare tutti e 3 i pianeti (Tatooine-Coruscant-Alderaan)"

In [7]:
def task3(droid="R2-D2", planets=("Tatooine", "Coruscant", "Alderaan")):
    from itertools import permutations  # Importa funzione per generare permutazioni

    intermediate_steps = []  # Per memorizzare i passaggi intermedi

    # Step 0: Ottieni la posizione attuale del droide dal galaxy_state
    start_location = find_droid(droid)

    if not start_location:
        return {
            "agent_response": f"Could not determine location of droid '{droid}'.",
            "intermediate_steps": ["Failed to retrieve droid location."],
            "final_state": {
                "status": "droid_location_unknown",
                "cheapest_route": None,
                "total_cost": None
            }
        }

    intermediate_steps.append(f"Droid '{droid}' is currently at {start_location}.")

    # Step 1: Genera tutte le permutazioni dei pianeti escluso start_location (che deve essere punto di partenza)
    # Filtro: prendo solo permutazioni che iniziano con start_location
    # Poiché start_location è punto di partenza, rimuovo start_location dalla lista e genero permutazioni degli altri
    planets_to_permute = [p for p in planets if p != start_location]

    # Genera tutte le permutazioni dei pianeti da visitare (escludendo start_location)
    subroutes = list(permutations(planets_to_permute))

    # Ricostruisco le rotte inserendo start_location come primo pianeta in ogni permutazione
    routes = [(start_location,) + route for route in subroutes]

    intermediate_steps.append(f"Generated {len(routes)} possible routes starting from {start_location}.")

    # Step 2: Calcola costo totale per ogni rotta
    route_costs = []
    for route in routes:
        total_cost = 0
        invalid_leg = False
        # Calcola i costi per ogni tratta della rotta
        for i in range(len(route) - 1):
            origin = route[i]
            destination = route[i + 1]
            leg_cost = infosphere.get_travel_cost(origin, destination)
            if leg_cost is None:
                invalid_leg = True
                break
            total_cost += leg_cost
        if not invalid_leg:
            route_costs.append({
                "route": route,
                "total_cost": total_cost
            })

    intermediate_steps.append("Calculated travel costs for all valid routes.")

    if not route_costs:
        agent_response = "Could not find any valid route connecting all planets starting from droid location."
        final_state = {
            "status": "route_unavailable",
            "cheapest_route": None,
            "total_cost": None
        }
    else:
        cheapest = min(route_costs, key=lambda x: x["total_cost"])
        agent_response = (
            "Identified the cheapest route: "
            + " -> ".join(cheapest["route"])
            + f" with total travel cost {cheapest['total_cost']} credits."
        )
        final_state = {
            "status": "success",
            "cheapest_route": list(cheapest["route"]),
            "total_cost": cheapest["total_cost"]
        }

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            (
                f"Cheapest route: {' -> '.join(cheapest['route'])} "
                f"with cost {cheapest['total_cost']}."
            ) if route_costs else "No valid routes available."
        ],
        "final_state": final_state
    }

In [8]:
#task3()

##### TASK 4: "Compra l'oggetto più costoso che puoi permetterti, poi usa i crediti rimanenti per comprare altri oggetti"

In [9]:
def task4():
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Get current budget
    budget = navigator.get_client_budget()
    intermediate_steps.append(f"Current budget: {budget} credits.")
    
    # Step 2: Find all available items and sort by price (descending)
    all_items = []
    planets = infosphere.get_all_planets()
    
    for planet in planets:
        planet_items = market.find_items(planet)
        all_items.extend(planet_items)
    
    # Filter items within budget and sort by price (most expensive first)
    affordable_items = [item for item in all_items if item["price"] <= budget]
    affordable_items.sort(key=lambda x: x["price"], reverse=True)
    
    intermediate_steps.append(f"Found {len(affordable_items)} affordable items across all planets.")
    
    if not affordable_items:
        agent_response = "No items affordable with current budget."
        final_state = {
            "status": "no_affordable_items",
            "purchases": [],
            "remaining_budget": budget
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }
    
    # Step 3: Purchase the most expensive item first
    most_expensive = affordable_items[0]
    most_expensive_planet = most_expensive["planet"]
    droid_location = find_droid("R2-D2")
    
    intermediate_steps.append(f"Most expensive affordable item: {most_expensive['name']} for {most_expensive['price']} credits on {most_expensive_planet}.")
    intermediate_steps.append(f"R2-D2 is currently at {droid_location}.")
    
    purchases = []
    errors = []
    total_spent = 0
    
    # Move droid to planet with most expensive item if needed
    if droid_location != most_expensive_planet:
        move_result = move_droid("R2-D2", most_expensive_planet)
        intermediate_steps.extend(move_result["intermediate_steps"])
        
        if move_result["final_state"]["status"] == "stuck":
            agent_response = f"Could not move droid to {most_expensive_planet}. Purchase failed."
            final_state = {
                "status": "move_failed",
                "purchases": [],
                "errors": [f"Could not reach {most_expensive_planet}"],
                "remaining_budget": budget
            }
            return {
                "agent_response": agent_response,
                "intermediate_steps": intermediate_steps,
                "final_state": final_state
            }
        
        droid_location = most_expensive_planet
    
    # Purchase most expensive item
    try:
        purchase_result = market.purchase_item(most_expensive["id"])
        purchases.append({
            "item_id": most_expensive["id"],
            "name": most_expensive["name"],
            "price": most_expensive["price"],
            "planet": most_expensive_planet,
            "purchase_info": purchase_result
        })
        total_spent += most_expensive["price"]
        intermediate_steps.append(f"Purchased most expensive item: {most_expensive['name']} for {most_expensive['price']} credits.")
    except ValueError as e:
        errors.append(f"Could not purchase {most_expensive['name']}: {str(e)}")
    
    # Step 4: Use remaining budget to buy additional items
    remaining_budget = navigator.get_client_budget()
    intermediate_steps.append(f"Remaining budget after most expensive purchase: {remaining_budget} credits.")
    
    # Get items from current planet first (to avoid travel costs)
    current_planet_items = [item for item in affordable_items[1:] if item["planet"] == droid_location and item["price"] <= remaining_budget]
    
    # Purchase items from current planet
    for item in current_planet_items:
        if navigator.get_client_budget() >= item["price"]:
            try:
                purchase_result = market.purchase_item(item["id"])
                purchases.append({
                    "item_id": item["id"],
                    "name": item["name"], 
                    "price": item["price"],
                    "planet": item["planet"],
                    "purchase_info": purchase_result
                })
                total_spent += item["price"]
                intermediate_steps.append(f"Purchased additional item: {item['name']} for {item['price']} credits.")
            except ValueError as e:
                errors.append(f"Could not purchase {item['name']}: {str(e)}")
    
    # If budget still available, consider items from other planets
    remaining_budget = navigator.get_client_budget()
    if remaining_budget > 0:
        other_planet_items = [item for item in affordable_items[1:] if item["planet"] != droid_location and item["price"] <= remaining_budget]
        other_planet_items.sort(key=lambda x: x["price"], reverse=True)
        
        for item in other_planet_items:
            current_budget = navigator.get_client_budget()
            if current_budget >= item["price"]:
                # Check if we can afford travel + item
                travel_cost = infosphere.get_travel_cost(droid_location, item["planet"])
                if travel_cost and current_budget >= (item["price"] + travel_cost):
                    # Move to planet
                    move_result = move_droid("R2-D2", item["planet"])
                    if move_result["final_state"]["status"] != "stuck":
                        try:
                            purchase_result = market.purchase_item(item["id"])
                            purchases.append({
                                "item_id": item["id"],
                                "name": item["name"],
                                "price": item["price"],
                                "planet": item["planet"],
                                "purchase_info": purchase_result
                            })
                            total_spent += item["price"]
                            droid_location = item["planet"]
                            intermediate_steps.append(f"Moved to {item['planet']} and purchased: {item['name']} for {item['price']} credits.")
                        except ValueError as e:
                            errors.append(f"Could not purchase {item['name']}: {str(e)}")
                        break  # Only try one additional planet to avoid excessive travel costs
    
    # Step 5: Final results
    final_budget = navigator.get_client_budget()
    
    if len(purchases) == 0:
        agent_response = "Could not purchase any items."
        status = "purchase_failed"
    elif len(purchases) == 1:
        agent_response = f"Successfully purchased the most expensive affordable item: {purchases[0]['name']}."
        status = "partial_success"
    else:
        agent_response = f"Successfully purchased {len(purchases)} items, starting with the most expensive affordable item."
        status = "success"
    
    final_state = {
        "status": status,
        "purchases": purchases,
        "errors": errors,
        "total_spent": budget - final_budget,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Total items purchased: {len(purchases)}.",
            f"Total spent: {budget - final_budget} credits.",
            f"Final budget: {final_budget} credits."
        ],
        "final_state": final_state
    }

In [10]:
#task4()

##### TASK 5: "Missione multi-step: porta R2-D2 su Alderaan, compra un Holocron, e torna su Tatooine"


In [11]:
def task5(droid="R2-D2", target_planet="Alderaan", target_item="Holocron", final_destination="Tatooine"):
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Check initial state
    initial_location = find_droid(droid)
    initial_budget = navigator.get_client_budget()
    intermediate_steps.append(f"Starting mission: {droid} is at {initial_location} with {initial_budget} credits.")
    
    errors = []
    purchases = []
    
    # Step 2: Move droid to target planet (Alderaan)
    if initial_location != target_planet:
        move_result = move_droid(droid, target_planet)
        intermediate_steps.extend(move_result["intermediate_steps"])
        
        if move_result["final_state"]["status"] == "stuck":
            agent_response = f"Mission failed: Could not move {droid} to {target_planet}."
            final_state = {
                "status": "move_failed",
                "step": "move_to_target",
                "droid_location": initial_location,
                "purchases": [],
                "errors": [f"Could not reach {target_planet}"],
                "remaining_budget": navigator.get_client_budget()
            }
            return {
                "agent_response": agent_response,
                "intermediate_steps": intermediate_steps,
                "final_state": final_state
            }
        
        intermediate_steps.append(f"Successfully moved {droid} to {target_planet}.")
    else:
        intermediate_steps.append(f"{droid} was already at {target_planet}.")
    
    current_location = target_planet
    
    # Step 3: Find and purchase Holocron
    items_on_planet = market.find_items(target_planet)
    holocron_items = [item for item in items_on_planet if target_item.lower() in item["name"].lower()]
    
    if not holocron_items:
        agent_response = f"Mission failed: No {target_item} found on {target_planet}."
        final_state = {
            "status": "item_not_found",
            "step": "purchase_item",
            "droid_location": current_location,
            "purchases": [],
            "errors": [f"No {target_item} available on {target_planet}"],
            "remaining_budget": navigator.get_client_budget()
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps + [f"Searched for {target_item} on {target_planet} - not found."],
            "final_state": final_state
        }
    
    # Try to purchase the first available Holocron
    holocron = holocron_items[0]
    current_budget = navigator.get_client_budget()
    
    if current_budget < holocron["price"]:
        agent_response = f"Mission failed: Insufficient funds to buy {holocron['name']} (costs {holocron['price']}, have {current_budget})."
        final_state = {
            "status": "insufficient_funds",
            "step": "purchase_item",
            "droid_location": current_location,
            "purchases": [],
            "errors": [f"Cannot afford {holocron['name']} - need {holocron['price']} credits, have {current_budget}"],
            "remaining_budget": current_budget
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps + [f"Found {holocron['name']} but cannot afford it."],
            "final_state": final_state
        }
    
    try:
        purchase_result = market.purchase_item(holocron["id"])
        purchases.append({
            "item_id": holocron["id"],
            "name": holocron["name"],
            "price": holocron["price"],
            "planet": target_planet,
            "purchase_info": purchase_result
        })
        intermediate_steps.append(f"Successfully purchased {holocron['name']} for {holocron['price']} credits.")
    except ValueError as e:
        agent_response = f"Mission failed: Could not purchase {holocron['name']}: {str(e)}"
        final_state = {
            "status": "purchase_failed",
            "step": "purchase_item",
            "droid_location": current_location,
            "purchases": [],
            "errors": [f"Purchase failed: {str(e)}"],
            "remaining_budget": navigator.get_client_budget()
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps + [f"Attempted to purchase {holocron['name']} but failed."],
            "final_state": final_state
        }
    
    # Step 4: Return to final destination (Tatooine)
    if current_location != final_destination:
        return_move = move_droid(droid, final_destination)
        intermediate_steps.extend(return_move["intermediate_steps"])
        
        if return_move["final_state"]["status"] == "stuck":
            agent_response = f"Partial success: Purchased {holocron['name']} but could not return to {final_destination}."
            final_state = {
                "status": "partial_success",
                "step": "return_move",
                "droid_location": current_location,
                "purchases": purchases,
                "errors": [f"Could not return to {final_destination}"],
                "remaining_budget": navigator.get_client_budget()
            }
            return {
                "agent_response": agent_response,
                "intermediate_steps": intermediate_steps,
                "final_state": final_state
            }
        
        intermediate_steps.append(f"Successfully returned {droid} to {final_destination}.")
        current_location = final_destination
    else:
        intermediate_steps.append(f"{droid} was already at {final_destination}.")
    
    # Step 5: Mission complete
    final_budget = navigator.get_client_budget()
    total_spent = initial_budget - final_budget
    
    agent_response = f"Mission completed successfully: Moved {droid} to {target_planet}, purchased {holocron['name']}, and returned to {final_destination}."
    
    final_state = {
        "status": "success",
        "step": "complete",
        "droid_location": current_location,
        "purchases": purchases,
        "errors": [],
        "total_spent": total_spent,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Mission summary: Purchased {len(purchases)} item(s).",
            f"Total spent: {total_spent} credits.",
            f"Final location: {current_location}.",
            f"Remaining budget: {final_budget} credits."
        ],
        "final_state": final_state
    }


In [12]:
#task5()

##### TASK 6: "Challenge finale: massimizza il numero di oggetti acquistati rimanendo nel budget di 5000 crediti"

In [13]:
def task6():
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Reset budget to 5000 credits
    navigator.galaxy_state["client"]["balance"] = 5000
    initial_budget = 5000
    intermediate_steps.append(f"Reset budget to {initial_budget} credits for final challenge.")
    
    # Step 2: Get initial droid location
    initial_location = find_droid("R2-D2")
    intermediate_steps.append(f"R2-D2 starting location: {initial_location}.")
    
    # Step 3: Find all available items and sort by price (cheapest first to maximize quantity)
    all_items = []
    planets = infosphere.get_all_planets()
    
    for planet in planets:
        planet_items = market.find_items(planet)
        all_items.extend(planet_items)
    
    # Sort by price (cheapest first) to maximize number of items
    all_items.sort(key=lambda x: x["price"])
    
    intermediate_steps.append(f"Found {len(all_items)} total items across all planets.")
    intermediate_steps.append(f"Strategy: Purchase cheapest items first to maximize quantity.")
    
    purchases = []
    errors = []
    total_spent = 0
    current_location = initial_location
    
    # Step 4: Greedily purchase items starting with cheapest
    for item in all_items:
        current_budget = navigator.get_client_budget()
        
        # Check if we can afford this item
        if current_budget < item["price"]:
            intermediate_steps.append(f"Cannot afford {item['name']} (costs {item['price']}, have {current_budget}). Skipping.")
            continue
        
        # Check if we need to travel to the item's planet
        if current_location != item["planet"]:
            travel_cost = infosphere.get_travel_cost(current_location, item["planet"])
            
            if travel_cost is None:
                errors.append(f"No travel route available from {current_location} to {item['planet']}")
                continue
            
            # Check if we can afford travel + item
            if current_budget < (item["price"] + travel_cost):
                intermediate_steps.append(f"Cannot afford travel to {item['planet']} + {item['name']} (total cost {item['price'] + travel_cost}, have {current_budget}). Skipping.")
                continue
            
            # Move to the planet
            move_result = move_droid("R2-D2", item["planet"])
            if move_result["final_state"]["status"] == "stuck":
                errors.append(f"Could not move to {item['planet']} for {item['name']}")
                continue
            
            current_location = item["planet"]
            intermediate_steps.append(f"Moved to {item['planet']} to purchase {item['name']}.")
        
        # Attempt to purchase the item
        try:
            purchase_result = market.purchase_item(item["id"])
            purchases.append({
                "item_id": item["id"],
                "name": item["name"],
                "price": item["price"],
                "planet": item["planet"],
                "purchase_info": purchase_result
            })
            total_spent += item["price"]
            intermediate_steps.append(f"Purchased {item['name']} for {item['price']} credits. (Total items: {len(purchases)})")
        except ValueError as e:
            errors.append(f"Could not purchase {item['name']}: {str(e)}")
            continue
    
    # Step 5: Calculate final results
    final_budget = navigator.get_client_budget()
    items_purchased = len(purchases)
    
    if items_purchased == 0:
        agent_response = "Challenge failed: Could not purchase any items."
        status = "failed"
    else:
        agent_response = f"Challenge completed: Successfully purchased {items_purchased} items within budget."
        status = "success"
    
    # Step 6: Analyze efficiency
    efficiency_stats = {
        "total_items_purchased": items_purchased,
        "total_spent": total_spent,
        "budget_remaining": final_budget,
        "budget_utilization": f"{((total_spent / initial_budget) * 100):.1f}%",
        "average_item_cost": round(total_spent / items_purchased, 2) if items_purchased > 0 else 0
    }
    
    final_state = {
        "status": status,
        "purchases": purchases,
        "errors": errors,
        "efficiency_stats": efficiency_stats,
        "final_droid_location": current_location,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Final challenge results:",
            f"- Items purchased: {items_purchased}",
            f"- Total spent: {total_spent} credits",
            f"- Budget remaining: {final_budget} credits",
            f"- Budget utilization: {efficiency_stats['budget_utilization']}",
            f"- Average cost per item: {efficiency_stats['average_item_cost']} credits",
            f"- Final droid location: {current_location}"
        ],
        "final_state": final_state
    }


In [14]:
#task6()

#### EVALUATION round 2

In [15]:
import json

# Esegui i task
result1 = task1()
result2 = task2()
result3 = task3()
result4 = task4()
result5 = task5()
result6 = task6()

# Formatta i risultati come richiesto
formatted_results = [
    {
        "task_id": "1",
        "agent_response": result1["agent_response"],
        "intermediate_steps": result1["intermediate_steps"],
        "final_state": result1["final_state"]
    },
    {
        "task_id": "2",
        "agent_response": result2["agent_response"],
        "intermediate_steps": result2["intermediate_steps"],
        "final_state": result2["final_state"]
    },
    {
        "task_id": "3",
        "agent_response": result3["agent_response"],
        "intermediate_steps": result3["intermediate_steps"],
        "final_state": result3["final_state"]
    },
    {
        "task_id": "4",
        "agent_response": result4["agent_response"],
        "intermediate_steps": result4["intermediate_steps"],
        "final_state": result4["final_state"]
    },
    {
        "task_id": "5",
        "agent_response": result5["agent_response"],
        "intermediate_steps": result5["intermediate_steps"],
        "final_state": result5["final_state"]
    },
    {
        "task_id": "6",
        "agent_response": result6["agent_response"],
        "intermediate_steps": result6["intermediate_steps"],
        "final_state": result6["final_state"]
    }
]

# Stampa JSON leggibile
print(json.dumps(formatted_results, indent=2, ensure_ascii=False))

# Salva su file
with open("round2_tasks.json", "w", encoding="utf-8") as f:
    json.dump(formatted_results, f, indent=2, ensure_ascii=False)

4500
Muoviamo il droide
Selected ship: StarHopper
Selected ship location: Tatooine
Destination: Coruscant
Total cost: 520
Client balance: 4500
{'agent_response': 'Travel successful: droid R2-D2 moved to Coruscant.', 'intermediate_steps': ['Located R2-D2 at Tatooine.', 'Booked travel from Tatooine to Coruscant.', 'Moved R2-D2 to Coruscant.'], 'final_state': {'droid': 'R2-D2', 'location': 'Coruscant', 'status': 'moved'}}
Selected ship: StarHopper
Selected ship location: Coruscant
Destination: Alderaan
Total cost: 500
Client balance: 3480
Selected ship: X-Wing
Selected ship location: Alderaan
Destination: Coruscant
Total cost: 400
Client balance: 2980
Selected ship: X-Wing
Selected ship location: Coruscant
Destination: Tatooine
Total cost: 420
Client balance: 580
Selected ship: X-Wing
Selected ship location: Tatooine
Destination: Coruscant
Total cost: 420
Client balance: 4750
Selected ship: X-Wing
Selected ship location: Coruscant
Destination: Alderaan
Total cost: 400
Client balance: 4030